## First KDE, Home made Empirical PMF, Experiment Simulation, Intuition of param changes

- Create an my own empirical PMF function 
    - to count the number of successes of a n trials.
    
- First Exercise and plot is a simulation how changing p changes the plot for binomial data.
- 2nd Exercise 
    - Create a KDE that shows how the changes in parameters changes the shape of the KDE.
    - Gain intuition on how param changes affect the experiment outcome
        - number of experiments
        - number of trials
        - sample size
    
    

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import math

from bokeh.io import output_notebook, show, output_file, push_notebook
from bokeh.plotting import figure


# For dynamic interactive data as a dictionary
from bokeh.models import ColumnDataSource

# Widget to accept inputs to modify the graphs. 
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Dropdown, Slider
from ipywidgets import interact
output_notebook()

Loading BokehJS ...

In [2]:
# actualize 10 random variables from. 
# try number_RVs experiments and repeat the process for 10 times, and return the k times it was a success in group of 10.
number_RVs = 50
number_repeat_experiments=10000
number_of_binom_trials = 10
chance_of_trial_success = .1
sim_data = stats.binom.rvs(size=[number_repeat_experiments, number_RVs], p=chance_of_trial_success, n=number_of_binom_trials)

# n - nuber of tries for 1 binomial RV (1 try of the 10 is a *bernoulli* RV)
# number_RVs number of binomial RVs instantiated in a row. 
# number_repeat_experiments is the number o sets of 5 of binomial RVs instantiated

# this is a simulation set, because it is hypothetical dummy data of theoretical distribution data. 
# We can eventually go out and get actual data. 

In [3]:
sim_data[0,:]

array([2, 0, 0, 1, 1, 1, 0, 1, 1, 2, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 0,
       0, 3, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 2, 3, 1, 0, 4, 0, 0, 3,
       0, 2, 0, 0])

In [4]:

def get_ePMF(sample, my_min=0, my_max=0):
    '''returns array of length max-min+1
    which counts how many of each outcome
    (as given by the index) were observed'''
    
    index = np.arange(my_min, my_max+1)
    my_ePMF = 0*index
    for i in index:
        my_ePMF[i] = np.sum(sample==i)
    
    return my_ePMF
    
samp = sim_data[0,:]    
get_ePMF(samp, my_max=number_of_binom_trials)

array([22, 19,  5,  3,  1,  0,  0,  0,  0,  0,  0])

In [5]:
#create circle plot with bokeh
#support is arange(0-10)
support = np.arange(11)
samp = sim_data[0,:]  

empirical_pmf = get_ePMF(samp, my_max=number_of_binom_trials)  #stats.binom.cdf(support, n, p)
#the one based on data we have. 

#make bokeh glyph

p = figure(title="first", plot_height=300, plot_width=600, y_range=[0,50])
r = p.circle(support, empirical_pmf, color='blue', line_width=3)


In [6]:
def update(p=.1):
    chance_of_trial_success = p    
    sim_data = stats.binom.rvs(size=[number_repeat_experiments, number_RVs], p=chance_of_trial_success, n=number_of_binom_trials)
    samp = sim_data[0,:] 
    empirical_pmf = get_ePMF(samp, my_max=number_of_binom_trials)
    r.data_source.data['x'] = support
    r.data_source.data['y'] = empirical_pmf
#     r = p.circle(support, empirical_pmf, color='blue', line_width=3)
    push_notebook()

In [7]:
show(p, notebook_handle=True)

In [8]:
interact(update, p=(.01,.99,.01))

<function __main__.update>

In [9]:
number_RVs = 50
number_repeat_experiments=10000
number_of_binom_trials = 10
chance_of_trial_success = .1
sim_data = stats.binom.rvs(size=[number_repeat_experiments, number_RVs], 
                           p=chance_of_trial_success, n=number_of_binom_trials)

In [10]:
from scipy.stats import gaussian_kde

In [11]:
averages = sim_data.mean(axis=1)
grid = np.linspace(0, number_of_binom_trials, 1000)
kde_of_averages = gaussian_kde(averages, bw_method=.1).evaluate(grid)

p = figure(title="first", plot_height=300, plot_width=600, y_range=[0,5], x_range=(0,10))
r = p.line(grid, kde_of_averages, color='blue', line_width=3)
r2 = p.circle(averages, 0*averages, color='red', line_width=3)

def update(number_RVs=50, number_repeat_experiments = 1000, 
         number_of_binom_trials =10, chance_of_trial_success = .1):

    sim_data = stats.binom.rvs(size=[number_repeat_experiments, number_RVs], 
                               p=chance_of_trial_success, n=number_of_binom_trials)

    averages = sim_data.mean(axis=1)
    kde_of_averages = gaussian_kde(averages, bw_method=.1).evaluate(grid)
    
    r.data_source.data['x'] = grid
    r.data_source.data['y'] = gaussian_kde(averages, bw_method=1).evaluate(grid)

    r2.data_source.data['x'] = averages
    r2.data_source.data['y'] = 0*averages

    push_notebook()

In [12]:
show(p, notebook_handle=True)

In [14]:
interact(update, number_RVs=(1,500), number_repeat_experiments = (1,1000), 
         number_of_binom_trials = (5,25), chance_of_trial_success=(.01,.99,.01))



<function __main__.update>

- number trials (and p for each trial)
- number of random variables (binomial)
- number or repeated experiments
    - turn each repeated experiment and made an average
- KDE (kernel density estimate) (continuous analog of a historgram) of the collection of averages (the distribution of averages)
    - bw (bandwidth parameter)
